In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
from fashion_data_module import FashionDataModule 
import wandb
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics.functional as tf

In [3]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.1.1+cu118  Device: cuda


In [4]:
param_dict = {
    "conv_out_dim":512,
    "hidden_dim":256, 
    "batch_size":32,
    "image_dim":224, 
    "learning_rate":0.0001,
    "momentum":0.9,
    "weight_decay":0.01, 
    "n_classes":5,
    "thresh":0.5,
    "use_cutmix":True,
    "use_pos_encoding":False,
    "epochs":30
}

wandb.init(project='temp',name='seorang_resnet')
wandb.run.save()
wandb.config.update(param_dict)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mongcha33 (uos_seorang). Use `wandb login --relogin` to force relogin


wandb: WARNING Calling wandb.run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


In [5]:
img_data_dir = "C:/Users/ksrks/OneDrive - UOS/문서 - 인공지능 프로젝트/fashion_dataset_v2/"

# FashionDataModule 사용
coco = FashionDataModule(
    img_data_dir,
    img_size=param_dict["image_dim"],
    batch_size=param_dict["batch_size"],
    use_cutmix=param_dict["use_cutmix"],
    cutmix_alpha=1.0)

coco.setup() # Creates train, validation, test datasets
param_dict["data"] = coco

train_loader = coco.train_dataloader() 
test_loader = coco.test_dataloader()
val_loader = coco.val_dataloader() 

print(param_dict["data"]) 

In [6]:
# CNN 정의[1] 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 28 * 28, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 5)
        self.dropout = nn.Dropout(0.5)  

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(-1, 32 * 28 * 28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x) 
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)  
        x = self.fc3(x)
        x = F.log_softmax(x) 

        return x

In [7]:
model = CNN().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr = param_dict['learning_rate']) # optimizer Adam
criterion = nn.CrossEntropyLoss() # 다중분류 손실함수

print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=25088, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=5, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [8]:
sum_of_rmap = 0.0
count = 1.0

In [9]:
# train 함수[1]
def train(model, train_loader, optimizer):
    
    model.train() 
    train_loss = 0 
    correct = 0 
    total_samples = 0

    for image, label, img_name in train_loader:
        image = image.to(DEVICE) # 이미지 
        label = label[0].to(DEVICE) # 라벨

        optimizer.zero_grad() 
        output = model(image) # 모델에 이미지 넣었을 때 결과(즉, 예측값)

        labels_indices = torch.argmax(label, dim=1) # 손실함수 계산을 위해 원-핫 벡터에서 클래스 인덱스로 변환 
        loss = criterion(output, label) # 손실 계산
        train_loss += loss.item()
         
        loss.backward() 
        optimizer.step() 

        predicted_labels = torch.argmax(output, dim=1) # correct 계산 위해 모델의 예측 클래스 인덱스로 변환
        correct += (predicted_labels == labels_indices).sum().item() # 예측과 정답이 같으면 correct 증가
        total_samples += label.size(0)
    
        
    train_loss /= len(train_loader) 
    train_accuracy = correct / total_samples
    wandb.log({"train_loss": train_loss, "train_accuracy":train_accuracy})
    return train_loss, train_accuracy

In [10]:
def val(model, test_loader, epoch): 
    model.eval()
    sum_of_rmap = 0.0
    test_loss = 0
    correct = 0
    total_samples = 0
    count = 1.0
    with torch.no_grad():
        for image, label, img_name in test_loader:
            image = image.to(DEVICE) # 이미지
            label= label.to(DEVICE) # 라벨

            output = model(image) # 모델에 이미지 넣었을 때 결과(즉, 예측값)

            labels_indices = torch.argmax(label, dim=1) # 손실함수 계산을 위해 원-핫 벡터에서 클래스 인덱스로 변환 
            loss = criterion(output, label) # 손실 계산
            test_loss += loss.item()

            predicted_labels = torch.argmax(output, dim=1) # mAP, correct 계산 위해 모델의 예측 클래스 인덱스로 변환

            one_hot_encoding = torch.zeros_like(output) 
            one_hot_encoding.scatter_(1, predicted_labels.view(-1, 1), 1) # mAP 계산 위해 output을 원-핫 인코딩4

            correct += (predicted_labels == labels_indices).sum().item() # 예측값 정답이 같으면 correct 증가
            total_samples += label.size(0)
            count += 1.0 # count 증가
            rmap = tf.retrieval_average_precision(one_hot_encoding, label) # mAP 계산
            sum_of_rmap = sum_of_rmap+float(rmap) # 반복에 대한 mAP 값을 누적

    test_loss /= len(test_loader) 
    test_accuracy = correct / total_samples
    wandb.log({"val loss": test_loss, "val accuracy":test_accuracy, "mAP":float(sum_of_rmap)/float(count)},step=epoch)
    return test_loss, test_accuracy, float(sum_of_rmap)/float(count)

In [11]:
def test(model, test_loader): 
    
    sum_of_rmap = 0.0
    
    count = 1.0
    model.eval()
    test_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for image, label, img_name in test_loader:
            image = image.to(DEVICE) # 이미지
            label= label.to(DEVICE) # 라벨
            print(label.shape)

            output = model(image) # 모델에 이미지 넣었을 때 결과(즉, 예측값)

            labels_indices = torch.argmax(label, dim=1) # 손실함수 계산을 위해 원-핫 벡터에서 클래스 인덱스로 변환 
            loss = criterion(output, label) # 손실 계산
            test_loss += loss.item()

            predicted_labels = torch.argmax(output, dim=1) # mAP, correct 계산 위해 모델의 예측 클래스 인덱스로 변환

            one_hot_encoding = torch.zeros_like(output) 
            one_hot_encoding.scatter_(1, predicted_labels.view(-1, 1), 1) # mAP 계산 위해 output을 원-핫 인코딩4

            correct += (predicted_labels == labels_indices).sum().item() # 예측값 정답이 같으면 correct 증가
            total_samples += label.size(0)

            rmap = tf.retrieval_average_precision(one_hot_encoding, label) # mAP 계산
            sum_of_rmap = sum_of_rmap+float(rmap) # 반복에 대한 mAP 값을 누적
            wandb.log({"total mAP": float(sum_of_rmap)/float(count)})
            count += 1.0
            
    test_loss /= len(test_loader) 
    test_accuracy = correct / total_samples
    wandb.log({"test loss": test_loss, "test accuracy":test_accuracy})
    
    return test_loss, test_accuracy, float(sum_of_rmap)/float(count)

In [12]:
sum_of_rmap = 0.0

for epoch in range(1, param_dict['epochs'] + 1): # 에폭만큼 반복하며 학습
    train_loss, train_accuracy = train(model, train_loader, optimizer)
    valid_loss, valid_accuracy, rmap = val(model, val_loader, epoch)

    print("\n[EPOCH: {}] Train Loss: {:.4f}, Train Accuracy: {:.2f} \n"
          "           Val Loss: {:.4f}, Val Accuracy: {:.2f}  \n".format(
        epoch, train_loss, train_accuracy, valid_loss, valid_accuracy))
    print("rmap:", rmap)
    
torch.save(model.state_dict(), 'model_weights.pth')
end_time = time.time()
wandb.finish()

C:\Users\ksrks\AppData\Local\Temp\ipykernel_22276\2196231548.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)



[EPOCH: 1] Train Loss: 1.6225, Train Accuracy: 0.27 
           Val Loss: 1.6105, Val Accuracy: 0.20  

rmap: 0.25782585516572

[EPOCH: 2] Train Loss: 1.5961, Train Accuracy: 0.29 
           Val Loss: 1.6184, Val Accuracy: 0.20  

rmap: 0.25782585516572

[EPOCH: 3] Train Loss: 1.5939, Train Accuracy: 0.29 
           Val Loss: 1.6101, Val Accuracy: 0.20  

rmap: 0.25782585516572

[EPOCH: 4] Train Loss: 1.5935, Train Accuracy: 0.29 
           Val Loss: 1.5945, Val Accuracy: 0.21  

rmap: 0.260276610031724

[EPOCH: 5] Train Loss: 1.5904, Train Accuracy: 0.29 
           Val Loss: 1.6070, Val Accuracy: 0.20  

rmap: 0.25782585516572

[EPOCH: 6] Train Loss: 1.5903, Train Accuracy: 0.30 
           Val Loss: 1.5913, Val Accuracy: 0.21  

rmap: 0.2518010772764683

[EPOCH: 7] Train Loss: 1.5842, Train Accuracy: 0.29 
           Val Loss: 1.5932, Val Accuracy: 0.22  

rmap: 0.2648552767932415

[EPOCH: 8] Train Loss: 1.5935, Train Accuracy: 0.29 
           Val Loss: 1.6047, Val Accuracy: 0.

mAP,▄▄▄▄▄▃▅▄▁▃▄▅▅▆▂▃▄▃▆▇▆▆▃▆▇▆█▇▅▅
train_accuracy,▁▄▄▄▄▆▄▅▇▅▅▄▆▅▆▆▆▅▆█▆▇▇▅▆▇▇█▇█
train_loss,█▆▅▅▅▅▄▅▄▅▅▅▄▃▃▄▃▃▃▄▃▃▂▄▃▂▂▁▂▂
val accuracy,▁▁▁▂▁▂▄▂▃▃▂▃▄▆▄▄▄▃▇▇▇▅▄██▇██▄▅
val loss,▇█▇▆▇▆▆▇▄▆▇▆▆▄▄▄▄▄▂▃▂▃▄▁▁▂▁▂▂▂
mAP,0.26545
train_accuracy,0.3142
train_loss,1.56278
val accuracy,0.22897
val loss,1.55219


In [13]:
wandb.init(project='temp-test',name='seorang_resnet')
wandb.run.save()
wandb.config.update(param_dict)
sum_of_rmap = 0.0

model.load_state_dict(torch.load('model_weights.pth'))
model.to(DEVICE) 
test_loss, test_accuracy, rmap = test(model, test_loader)

torch.Size([32, 5])
torch.Size([32, 5])


C:\Users\ksrks\AppData\Local\Temp\ipykernel_22276\2196231548.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([32, 5])
torch.Size([13, 5])


In [14]:
'''
참고 문헌

[1]https://github.com/Justin-A/DeepLearning101
'''

'\n참고 문헌\n\n[1]https://github.com/Justin-A/DeepLearning101\n'